# Venue data of cities from Foursquare API

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import os
pd.options.display.max_columns = 500

In [4]:
# os.getenv('PATH')

In [5]:
# os.getenv('PWD')

## Foursquare API test with Berlin

In [6]:
# run source CREDENTIALS command before jupyter notebook

In [7]:
client_id = os.getenv('FOURSQUARE_CLIENT_ID')
client_secret = os.getenv('FOURSQUARE_CLIENT_SECRET')
version = '20180323'

In [9]:
test_limit = 100

In [10]:
# Foursquare API call example - WORKS

url = 'https://api.foursquare.com/v2/venues/explore'

params = (dict(
    client_id=client_id,
    client_secret=client_secret,
    v='20180323',
    #ll='40.7243,-74.0018',
    near='Berlin',
    #query='coffee',
    limit=test_limit
))

resp = requests.get(url=url, params=params)
data = json.loads(resp.text)

In [11]:
#resp.json()

In [12]:
venues = resp.json()

In [13]:
n_results = venues['response']['totalResults']
n_results

219

In [14]:
v_name = venues['response']['groups'][0]['items'][11]['venue']['name']
v_name

'Park am Gleisdreieck - Ostpark'

In [15]:
v_cat_name = venues['response']['groups'][0]['items'][11]['venue']['categories'][0]['name']
v_cat_name

'Park'

In [16]:
len(range(test_limit))

100

In [17]:
v_list = []

for i in range(test_limit):
    v_name = venues['response']['groups'][0]['items'][i]['venue']['name']
    v_cat_name = venues['response']['groups'][0]['items'][i]['venue']['categories'][0]['name']
    v_list.append((v_name, v_cat_name))

In [18]:
v_list[0:9]

[('Tiergarten', 'Park'),
 ('Landwehrkanal', 'Canal'),
 ('Urban Nation', 'Art Gallery'),
 ('Tempelhofer Feld', 'Park'),
 ('Café Komine', 'Café'),
 ('Treptower Park', 'Park'),
 ('Classic Remise Berlin', 'Museum'),
 ('Das Stue', 'Hotel'),
 ('Viktoria-Luise-Platz', 'Plaza')]

In [19]:
v_name_list = []
for j in range(test_limit):
    v_name = venues['response']['groups'][0]['items'][j]['venue']['name']
    v_name_list.append(v_name)

In [20]:
v_name_list[0:9]

['Tiergarten',
 'Landwehrkanal',
 'Urban Nation',
 'Tempelhofer Feld',
 'Café Komine',
 'Treptower Park',
 'Classic Remise Berlin',
 'Das Stue',
 'Viktoria-Luise-Platz']

In [21]:
v_cat_name_list = []
for k in range(test_limit):
    v_cat_name = venues['response']['groups'][0]['items'][k]['venue']['categories'][0]['name']
    v_cat_name_list.append(v_cat_name)

In [22]:
v_cat_name_list[0:9]

['Park',
 'Canal',
 'Art Gallery',
 'Park',
 'Café',
 'Park',
 'Museum',
 'Hotel',
 'Plaza']

In [23]:
count = [[x, v_cat_name_list.count(x)] for x in set(v_cat_name_list)]

In [24]:
sorted(count)[0:9]

[['Art Gallery', 2],
 ['Art Museum', 1],
 ['Arts & Crafts Store', 1],
 ['Austrian Restaurant', 1],
 ['Bakery', 5],
 ['Bar', 3],
 ['Beer Bar', 2],
 ['Beer Garden', 1],
 ['Beer Store', 1]]

## Create a test DataFrame for Berlin

In [25]:
dict_test = {'city': 'berlin', 
             'venue_name': v_name_list, 
             'venue_category': v_cat_name_list}

In [26]:
#dict_test

In [27]:
pd.DataFrame(dict_test).head()

,city,venue_name,venue_category
0,berlin,Tiergarten,Park
1,berlin,Landwehrkanal,Canal
2,berlin,Urban Nation,Art Gallery
3,berlin,Tempelhofer Feld,Park
4,berlin,Café Komine,Café


## Create a test function for Berlin

In [28]:
def venues_of_a_city(city, lim=100):
    
    '''This function collects the venues of a city from the Foursquare API
    and returns the results as a Pandas DataFrame'''
    
    # Foursquare API
    client_id = os.getenv('FOURSQUARE_CLIENT_ID')
    client_secret = os.getenv('FOURSQUARE_CLIENT_SECRET')
    version = '20180323'
    
    url = 'https://api.foursquare.com/v2/venues/explore'

    params = (dict(
        client_id=client_id,
        client_secret=client_secret,
        v=version,
        near=city,
        limit=lim
    ))
    
    resp = requests.get(url=url, params=params)
    
    venues = resp.json()
    
    # get the venue names
    v_name_list = []
    for j in range(lim):
        v_name = venues['response']['groups'][0]['items'][j]['venue']['name']
        v_name_list.append(v_name)
    
    # get the venue categories
    v_cat_name_list = []
    for k in range(lim):
        v_cat_name = venues['response']['groups'][0]['items'][k]['venue']['categories'][0]['name']
        v_cat_name_list.append(v_cat_name)
    
    # create a dictionary
    dict_berlin = {'city': city, 
                   'venue_name': v_name_list, 
                   'venue_category': v_cat_name_list}
    
    # create a DateFrame
    df_berlin = pd.DataFrame(dict_berlin)
    
    return df_berlin


In [29]:
df_berlin = venues_of_a_city('Berlin')
df_berlin.head()

,city,venue_name,venue_category
0,Berlin,Tiergarten,Park
1,Berlin,Landwehrkanal,Canal
2,Berlin,Urban Nation,Art Gallery
3,Berlin,Tempelhofer Feld,Park
4,Berlin,Café Komine,Café


## Transform the test df

In [30]:
berlin_dummies = pd.get_dummies(df_berlin[['venue_category']])
berlin_dummies['city'] = df_berlin['city']
cols = list(berlin_dummies.columns)
cols = [cols[-1]] + cols[:-1]
berlin_dummies = berlin_dummies[cols]
berlin_dummies.head()

,city,venue_category_Art Gallery,venue_category_Art Museum,venue_category_Arts & Crafts Store,venue_category_Austrian Restaurant,venue_category_Bakery,venue_category_Bar,venue_category_Beer Bar,venue_category_Beer Garden,venue_category_Beer Store,venue_category_Bistro,venue_category_Bookstore,venue_category_Breakfast Spot,venue_category_Café,venue_category_Canal,venue_category_Caucasian Restaurant,venue_category_Climbing Gym,venue_category_Cocktail Bar,venue_category_Coffee Shop,venue_category_Concert Hall,venue_category_Event Space,venue_category_Falafel Restaurant,venue_category_Farmers Market,venue_category_Food Court,venue_category_Forest,venue_category_French Restaurant,venue_category_Gourmet Shop,venue_category_Hostel,venue_category_Hotel,venue_category_Ice Cream Shop,venue_category_Indie Movie Theater,venue_category_Indie Theater,venue_category_Juice Bar,venue_category_Karaoke Bar,venue_category_Korean Restaurant,venue_category_Mediterranean Restaurant,venue_category_Monument / Landmark,venue_category_Mountain,venue_category_Museum,venue_category_Music Venue,venue_category_Park,venue_category_Pizza Place,venue_category_Plaza,venue_category_Scenic Lookout,venue_category_Spa,venue_category_Spanish Restaurant,venue_category_Vegetarian / Vegan Restaurant,venue_category_Whisky Bar,venue_category_Wine Bar
0,Berlin,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,Berlin,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Berlin,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Berlin,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,Berlin,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [31]:
berlin_dummies.shape

(100, 49)

In [32]:
berlin_dummies['venue_category_Park'].sum()

13

## Define a function that collects all the venues of the capital cities of Europe

In [33]:
def venue_collector(lst, lim=100):
    
    '''This function collects the venues of a list of cities from the Foursquare API
    and returns the results as a Pandas DataFrame'''
    
    # Foursquare API
    client_id = os.getenv('FOURSQUARE_CLIENT_ID')
    client_secret = os.getenv('FOURSQUARE_CLIENT_SECRET')
    version = '20180323'
    
    list_of_dfs = []
    
    for city in lst:
        
        url = 'https://api.foursquare.com/v2/venues/explore'

        params = (dict(
            client_id=client_id,
            client_secret=client_secret,
            v=version,
            near=city,
            limit=lim
        ))
    
        resp = requests.get(url=url, params=params)
    
        venues = resp.json()
        
        # number of results of the API response
        n_results = venues['response']['totalResults']
    
        # get the venue names
        v_name_list = []
        for j in range(lim if lim < n_results else n_results): # use the num of results if its less than the limit
            v_name = venues['response']['groups'][0]['items'][j]['venue']['name']
            v_name_list.append(v_name)
    
        # get the venue categories
        v_cat_name_list = []
        for k in range(lim if lim < n_results else n_results):
            v_cat_name = venues['response']['groups'][0]['items'][k]['venue']['categories'][0]['name']
            v_cat_name_list.append(v_cat_name)
    
        # create a dictionary
        dict_city = {'city': city, 
                       'venue_name': v_name_list, 
                       'venue_category': v_cat_name_list}
    
        # create a DateFrame
        df_city = pd.DataFrame(dict_city)
        
        # append it the the list of DataFrames
        list_of_dfs.append(df_city)
    
    # concatenate all DataFrames
    df = pd.concat(list_of_dfs).reset_index(drop=True)
    
    return df

## Test the function

In [34]:
test_cap_cities_europe = ['Amsterdam',
                          'Athens',
                          'Andorra la Vella',
                          'Belgrade',
                          'Berlin',
                          'Bern']

In [35]:
test_cap_venues = venue_collector(test_cap_cities_europe)

test_cap_venues.head()

,city,venue_name,venue_category
0,Amsterdam,Vondelpark,Park
1,Amsterdam,Amsterdamse Grachten,Canal
2,Amsterdam,Concerto Records,Record Shop
3,Amsterdam,Zoku,Hotel
4,Amsterdam,Margherita Tutta La Vita!,Pizza Place


## Run the function on all the capital cities of Europe

In [39]:
cap_cities_europe = ['Amsterdam',
                     'Andorra la Vella',
                     'Athens',
                     'Belgrade',
                     'Berlin',
                     'Bern',
                     'Bratislava',
                     'Brussels',
                     'Bucharest',
                     'Budapest',
                     'Chisinau',
                     'Copenhagen',
                     'Dublin',
                     'Helsinki',
                     'Kiev',
                     'Lisbon',
                     'Ljubljana',
                     'London',
                     'Luxembourg',
                     'Madrid',
                     'Minsk',
                     'Monaco',
                     'Moscow',
                     'Nicosia',
                     'Nuuk',
                     'Oslo',
                     'Paris',
                     'Podgorica',
                     'Prague',
                     'Reykjavik',
                     'Riga',
                     'Rome',
                     'San Marino',
                     'Sarajevo',
                     'Skopje',
                     'Sofia',
                     'Stockholm',
                     'Tallinn',
                     'Tirana',
                     'Vaduz',
                     'Valletta',
                     'Vatican City',
                     'Vienna',
                     'Vilnius',
                     'Warsaw',
                     'Zagreb'] # 'Zurich'

In [40]:
len(cap_cities_europe)

46

In [41]:
cap_cities_europe_venues = venue_collector(cap_cities_europe)

In [42]:
cap_cities_europe_venues.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4311 entries, 0 to 4310
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   city            4311 non-null   object
 1   venue_name      4311 non-null   object
 2   venue_category  4311 non-null   object
dtypes: object(3)
memory usage: 101.2+ KB


In [43]:
cap_cities_europe_venues.head()

,city,venue_name,venue_category
0,Amsterdam,Vondelpark,Park
1,Amsterdam,Amsterdamse Grachten,Canal
2,Amsterdam,Concerto Records,Record Shop
3,Amsterdam,Zoku,Hotel
4,Amsterdam,Margherita Tutta La Vita!,Pizza Place


## Check and clear the venue categories

In [45]:
df1 = cap_cities_europe_venues.copy()

In [47]:
df1['venue_category'].value_counts()

Park               277
Café               201
Hotel              197
Coffee Shop        185
Restaurant         133
                  ... 
Waterfall            1
Aquarium             1
Auto Dealership      1
Auditorium           1
Pet Store            1
Name: venue_category, Length: 352, dtype: int64

In [67]:
df1['venue_category'].nunique()

352

In [48]:
df1['venue_category'].unique()

array(['Park', 'Canal', 'Record Shop', 'Hotel', 'Pizza Place', 'Hostel',
       'French Restaurant', 'Yoga Studio', 'Wine Bar', 'Cocktail Bar',
       'Concert Hall', 'Coffee Shop', 'Brewery', 'Breakfast Spot', 'Pub',
       'Garden', 'Bakery', 'Theater', 'Deli / Bodega', 'Dance Studio',
       'Distillery', 'Event Space', 'Plaza', 'Bookstore', 'Dessert Shop',
       'Sandwich Place', 'Turkish Restaurant', 'Museum', 'Gastropub',
       'Farmers Market', 'Bistro', 'Art Museum', 'Tour Provider',
       'Liquor Store', 'Music Venue', 'Café', 'Cycle Studio',
       'Monument / Landmark', 'Beer Bar', 'Caribbean Restaurant',
       'Multiplex', 'Grocery Store', 'Ice Cream Shop',
       'Falafel Restaurant', 'Restaurant', 'Japanese Restaurant', 'Bar',
       'Supermarket', 'Movie Theater', 'Marijuana Dispensary',
       'Italian Restaurant', 'Indie Movie Theater', 'Cheese Shop',
       'BBQ Joint', 'Design Studio', 'Comfort Food Restaurant', 'Arcade',
       'Lounge', 'Spanish Restaurant', 'P

**create a new column and standardize the categories**

In [94]:
df2 = df1.copy()
df2['venue_cat_standard'] = df2['venue_category']
df2.head()

,city,venue_name,venue_category,venue_cat_standard
0,Amsterdam,Vondelpark,Park,Park
1,Amsterdam,Amsterdamse Grachten,Canal,Canal
2,Amsterdam,Concerto Records,Record Shop,Record Shop
3,Amsterdam,Zoku,Hotel,Hotel
4,Amsterdam,Margherita Tutta La Vita!,Pizza Place,Pizza Place


In [95]:
# 1. Restaurant
df2.loc[df2['venue_cat_standard'].str.contains('Restaurant'), 'venue_cat_standard'] = 'Restaurant'

# 2. Museum
df2.loc[df2['venue_cat_standard'].str.contains('Museum'), 'venue_cat_standard'] = 'Museum'

# 3. Club
df2.loc[df2['venue_cat_standard'].str.contains('Club'), 'venue_cat_standard'] = 'Club'

# 3. Bar
df2.loc[df2['venue_cat_standard'].str.contains('Bar'), 'venue_cat_standard'] = 'Bar'

# 4. Cafe
df2.loc[df2['venue_cat_standard'].str.contains('Cafe'), 'venue_cat_standard'] = 'Café'

In [96]:
df2['venue_cat_standard'].value_counts()

Restaurant         767
Bar                341
Park               277
Café               205
Hotel              197
                  ... 
Auto Dealership      1
Stables              1
Swim School          1
Wings Joint          1
Pet Store            1
Name: venue_cat_standard, Length: 266, dtype: int64

In [97]:
df2['venue_cat_standard'].nunique()

266

In [98]:
df2['venue_cat_standard'].unique()

array(['Park', 'Canal', 'Record Shop', 'Hotel', 'Pizza Place', 'Hostel',
       'Restaurant', 'Yoga Studio', 'Bar', 'Concert Hall', 'Coffee Shop',
       'Brewery', 'Breakfast Spot', 'Pub', 'Garden', 'Bakery', 'Theater',
       'Deli / Bodega', 'Dance Studio', 'Distillery', 'Event Space',
       'Plaza', 'Bookstore', 'Dessert Shop', 'Sandwich Place', 'Museum',
       'Gastropub', 'Farmers Market', 'Bistro', 'Tour Provider',
       'Liquor Store', 'Music Venue', 'Café', 'Cycle Studio',
       'Monument / Landmark', 'Multiplex', 'Grocery Store',
       'Ice Cream Shop', 'Supermarket', 'Movie Theater',
       'Marijuana Dispensary', 'Indie Movie Theater', 'Cheese Shop',
       'BBQ Joint', 'Design Studio', 'Arcade', 'Lounge', 'Public Art',
       'Sporting Goods Shop', 'Spa', 'Resort', 'Basketball Court',
       'Stadium', 'Shopping Mall', 'Miscellaneous Shop', 'Historic Site',
       'Food & Drink Shop', 'Lingerie Store', 'Buffet', 'Gym',
       'Gym / Fitness Center', 'Electronics Store

In [99]:
df2.head()

,city,venue_name,venue_category,venue_cat_standard
0,Amsterdam,Vondelpark,Park,Park
1,Amsterdam,Amsterdamse Grachten,Canal,Canal
2,Amsterdam,Concerto Records,Record Shop,Record Shop
3,Amsterdam,Zoku,Hotel,Hotel
4,Amsterdam,Margherita Tutta La Vita!,Pizza Place,Pizza Place


## Group by cities and venue categories

In [100]:
df3 = df2.copy()

In [101]:
df3['venue_cat_count'] = 1

In [103]:
df3.head()

,city,venue_name,venue_category,venue_cat_standard,venue_cat_count
0,Amsterdam,Vondelpark,Park,Park,1
1,Amsterdam,Amsterdamse Grachten,Canal,Canal,1
2,Amsterdam,Concerto Records,Record Shop,Record Shop,1
3,Amsterdam,Zoku,Hotel,Hotel,1
4,Amsterdam,Margherita Tutta La Vita!,Pizza Place,Pizza Place,1


In [104]:
df3.isna().sum()

city                  0
venue_name            0
venue_category        0
venue_cat_standard    0
venue_cat_count       0
dtype: int64

In [105]:
df3['venue_category'].nunique()

352

In [106]:
df3['venue_cat_standard'].nunique()

266

In [108]:
df3 = df3.groupby(['city', 'venue_cat_standard']).agg({'venue_cat_count': 'sum'})

In [110]:
df3 = df3.reset_index()

In [111]:
df3.head()

,city,venue_cat_standard,venue_cat_count
0,Amsterdam,Arcade,1
1,Amsterdam,BBQ Joint,1
2,Amsterdam,Bakery,2
3,Amsterdam,Bar,8
4,Amsterdam,Bistro,1


**Check if the number of parks in Berlin is still 13**

In [113]:
df3[(df3['city'] == 'Berlin') & (df3['venue_cat_standard'] == 'Park')]

,city,venue_cat_standard,venue_cat_count
188,Berlin,Park,13


## Tranform the venue categories to columns

In [115]:
df4 = df3.copy()

In [116]:
df4 = df4.pivot(index='city', columns='venue_cat_standard', values='venue_cat_count')

In [117]:
df4.shape

(46, 266)

In [118]:
df4.head()

venue_cat_standard,Accessories Store,Adult Boutique,Airport,Antique Shop,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Athletics & Sports,Auditorium,Auto Dealership,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Ballroom,Bank,Bar,Basketball Court,Bathing Area,Beach,Bed & Breakfast,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bistro,Board Shop,Boarding House,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Butcher,Cable Car,Café,Camera Store,Canal,Canal Lock,Candy Store,Capitol Building,Casino,Castle,Cemetery,Cheese Shop,Chocolate Shop,Church,City Hall,Climbing Gym,Clothing Store,Club,Coffee Shop,College Arts Building,College Library,Comic Shop,Concert Hall,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distillery,Dog Run,Donut Shop,Drive-in Theater,Dry Cleaner,Electronics Store,Event Space,Exhibit,Farmers Market,Field,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Forest,Fountain,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Heliport,Hill,Historic Site,Hobby Shop,Hostel,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indie Movie Theater,Indie Theater,Irish Pub,Island,Jewelry Store,Kafenio,Lake,Laser Tag,Lawyer,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Massage Studio,Men's Store,Meyhane,Miscellaneous Shop,Mobile Phone Shop,Monastery,Monument / Landmark,Motel,Mountain,Movie Theater,Multiplex,Museum,Music Venue,National Park,Nature Preserve,Neighborhood,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Event Space,Outdoor Sculpture,Paintball Field,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Pelmeni House,Performing Arts Venue,Perfume Shop,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Racetrack,Record Shop,Recreation Center,Reservoir,Resort,Restaurant,Road,Roof Deck,Salad Place,Sandwich Place,Sauna / Steam Room,Sausage Shop,Scenic Lookout,Sculpture Garden,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Lodge,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Souvenir Shop,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Squash Court,Stables,Stadium,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Swim School,Taco Place,Tailor Shop,Tanning Salon,Tea Room,Temple,Tennis Court,Tennis Stadium,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Toy / Game Store,Track,Track Stadium,Trail,Trattoria/Osteria,Used Bookstore,Vineyard,Watch Shop,Water Park,Waterfall,Waterfront,Wine Shop,Winery,Wings Joint,Yoga Studio,Zoo,Zoo Exhibit
city,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Amsterdam,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2.0,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [119]:
df4 = df4.reset_index()

In [120]:
df4.head()

venue_cat_standard,city,Accessories Store,Adult Boutique,Airport,Antique Shop,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Athletics & Sports,Auditorium,Auto Dealership,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Ballroom,Bank,Bar,Basketball Court,Bathing Area,Beach,Bed & Breakfast,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bistro,Board Shop,Boarding House,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Butcher,Cable Car,Café,Camera Store,Canal,Canal Lock,Candy Store,Capitol Building,Casino,Castle,Cemetery,Cheese Shop,Chocolate Shop,Church,City Hall,Climbing Gym,Clothing Store,Club,Coffee Shop,College Arts Building,College Library,Comic Shop,Concert Hall,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distillery,Dog Run,Donut Shop,Drive-in Theater,Dry Cleaner,Electronics Store,Event Space,Exhibit,Farmers Market,Field,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Forest,Fountain,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Heliport,Hill,Historic Site,Hobby Shop,Hostel,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indie Movie Theater,Indie Theater,Irish Pub,Island,Jewelry Store,Kafenio,Lake,Laser Tag,Lawyer,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Massage Studio,Men's Store,Meyhane,Miscellaneous Shop,Mobile Phone Shop,Monastery,Monument / Landmark,Motel,Mountain,Movie Theater,Multiplex,Museum,Music Venue,National Park,Nature Preserve,Neighborhood,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Event Space,Outdoor Sculpture,Paintball Field,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Pelmeni House,Performing Arts Venue,Perfume Shop,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Racetrack,Record Shop,Recreation Center,Reservoir,Resort,Restaurant,Road,Roof Deck,Salad Place,Sandwich Place,Sauna / Steam Room,Sausage Shop,Scenic Lookout,Sculpture Garden,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Lodge,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Souvenir Shop,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Squash Court,Stables,Stadium,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Swim School,Taco Place,Tailor Shop,Tanning Salon,Tea Room,Temple,Tennis Court,Tennis Stadium,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Toy / Game Store,Track,Track Stadium,Trail,Trattoria/Osteria,Used Bookstore,Vineyard,Watch Shop,Water Park,Waterfall,Waterfront,Wine Shop,Winery,Wings Joint,Yoga Studio,Zoo,Zoo Exhibit
0,Amsterdam,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2.0,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,8.0,NaN,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN

In [121]:
df4.tail()

venue_cat_standard,city,Accessories Store,Adult Boutique,Airport,Antique Shop,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Athletics & Sports,Auditorium,Auto Dealership,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Ballroom,Bank,Bar,Basketball Court,Bathing Area,Beach,Bed & Breakfast,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bistro,Board Shop,Boarding House,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Butcher,Cable Car,Café,Camera Store,Canal,Canal Lock,Candy Store,Capitol Building,Casino,Castle,Cemetery,Cheese Shop,Chocolate Shop,Church,City Hall,Climbing Gym,Clothing Store,Club,Coffee Shop,College Arts Building,College Library,Comic Shop,Concert Hall,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distillery,Dog Run,Donut Shop,Drive-in Theater,Dry Cleaner,Electronics Store,Event Space,Exhibit,Farmers Market,Field,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Forest,Fountain,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Heliport,Hill,Historic Site,Hobby Shop,Hostel,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indie Movie Theater,Indie Theater,Irish Pub,Island,Jewelry Store,Kafenio,Lake,Laser Tag,Lawyer,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Massage Studio,Men's Store,Meyhane,Miscellaneous Shop,Mobile Phone Shop,Monastery,Monument / Landmark,Motel,Mountain,Movie Theater,Multiplex,Museum,Music Venue,National Park,Nature Preserve,Neighborhood,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Event Space,Outdoor Sculpture,Paintball Field,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Pelmeni House,Performing Arts Venue,Perfume Shop,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Racetrack,Record Shop,Recreation Center,Reservoir,Resort,Restaurant,Road,Roof Deck,Salad Place,Sandwich Place,Sauna / Steam Room,Sausage Shop,Scenic Lookout,Sculpture Garden,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Lodge,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Souvenir Shop,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Squash Court,Stables,Stadium,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Swim School,Taco Place,Tailor Shop,Tanning Salon,Tea Room,Temple,Tennis Court,Tennis Stadium,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Toy / Game Store,Track,Track Stadium,Trail,Trattoria/Osteria,Used Bookstore,Vineyard,Watch Shop,Water Park,Waterfall,Waterfront,Wine Shop,Winery,Wings Joint,Yoga Studio,Zoo,Zoo Exhibit
41,Vatican City,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,19.0,NaN,NaN,NaN,NaN,NaN,Na

In [123]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Columns: 267 entries, city to Zoo Exhibit
dtypes: float64(266), object(1)
memory usage: 96.1+ KB


## Export to JSON format

In [124]:
df4.to_json('../data/Foursquare_venues_data.json')

In [125]:
check = pd.read_json('../data/Foursquare_venues_data.json')
check.head()

,city,Accessories Store,Adult Boutique,Airport,Antique Shop,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Athletics & Sports,Auditorium,Auto Dealership,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Ballroom,Bank,Bar,Basketball Court,Bathing Area,Beach,Bed & Breakfast,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bistro,Board Shop,Boarding House,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Butcher,Cable Car,Café,Camera Store,Canal,Canal Lock,Candy Store,Capitol Building,Casino,Castle,Cemetery,Cheese Shop,Chocolate Shop,Church,City Hall,Climbing Gym,Clothing Store,Club,Coffee Shop,College Arts Building,College Library,Comic Shop,Concert Hall,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distillery,Dog Run,Donut Shop,Drive-in Theater,Dry Cleaner,Electronics Store,Event Space,Exhibit,Farmers Market,Field,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Forest,Fountain,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Heliport,Hill,Historic Site,Hobby Shop,Hostel,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indie Movie Theater,Indie Theater,Irish Pub,Island,Jewelry Store,Kafenio,Lake,Laser Tag,Lawyer,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Massage Studio,Men's Store,Meyhane,Miscellaneous Shop,Mobile Phone Shop,Monastery,Monument / Landmark,Motel,Mountain,Movie Theater,Multiplex,Museum,Music Venue,National Park,Nature Preserve,Neighborhood,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Event Space,Outdoor Sculpture,Paintball Field,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Pelmeni House,Performing Arts Venue,Perfume Shop,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Racetrack,Record Shop,Recreation Center,Reservoir,Resort,Restaurant,Road,Roof Deck,Salad Place,Sandwich Place,Sauna / Steam Room,Sausage Shop,Scenic Lookout,Sculpture Garden,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Lodge,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Souvenir Shop,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Squash Court,Stables,Stadium,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Swim School,Taco Place,Tailor Shop,Tanning Salon,Tea Room,Temple,Tennis Court,Tennis Stadium,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Toy / Game Store,Track,Track Stadium,Trail,Trattoria/Osteria,Used Bookstore,Vineyard,Watch Shop,Water Park,Waterfall,Waterfront,Wine Shop,Winery,Wings Joint,Yoga Studio,Zoo,Zoo Exhibit
0,Amsterdam,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2.0,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,8.0,NaN,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N